# Data Preparation

In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns

In [2]:
pal = 'deep'
st = 'whitegrid'
fig_size = {'figure.figsize':(10, 8)}

sns.set_theme(context='notebook',
              style= st,
              palette= pal,
              font='sans-serif',
              font_scale=1,
              color_codes=True,
              rc= fig_size)

In [3]:
df = pd.read_csv('../data/bd-recommendations.csv')

In [4]:
df.head()

,title,price,author,publisher,isbn,category,translator,pages,date,series,recommendations,bookstores,synopsis
0,Cambiare l'acqua ai fiori,18,Perrin Valérie,E/O,9788833570990,Letterature straniere: testi,Bracci Testasecca A.,476,10/07/2019,Dal mondo,63.0,"81/libreria-tasso, 60/a-tutto-libro, 55/libris...",\nTab Article\nViolette Toussaint è guardiana ...
1,"Ragazza, donna, altro",20,Evaristo Bernardine,Sur,9788869982279,Letterature straniere: testi,Testa M.,520,11/11/2020,BigSur,61.0,39/la-confraternita-delluva-libreria-indipende...,\nTab Article\nÈ una grande serata per Amma: u...
2,L'Ickabog,19,Rowling J. K.,Salani,9788831006705,Letteratura per ragazzi,Daniele V.,320,10/11/2020,NaN,44.0,"168/magia-del-fumetto, 486/lotta-libreria, 306...",\nTab Article\nC'era una volta un regno chiama...
3,Borgo Sud,18,Di Pietrantonio Donatella,Einaudi,9788806244781,Letteratura italiana: testi,NaN,168,03/11/2020,Supercoralli,36.0,"285/libreria-fogola-pisa, 512/libreria-di-donn...",\nTab Article\nÈ il momento più buio della not...
4,Dante,20,Barbero Alessandro,Laterza,9788858141649,Letteratura italiana: critica,NaN,361,08/10/2020,I Robinson,36.0,"51/il-ponte-sulla-dora, 116/lettera22, 433/lib...","\nTab Article\nDante è l'uomo su cui, per la f..."


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21014 entries, 0 to 21013
Data columns (total 13 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   title            21014 non-null  object 
 1   price            21014 non-null  int64  
 2   author           19732 non-null  object 
 3   publisher        21014 non-null  object 
 4   isbn             21014 non-null  int64  
 5   category         20995 non-null  object 
 6   translator       7648 non-null   object 
 7   pages            20021 non-null  object 
 8   date             21014 non-null  object 
 9   series           16087 non-null  object 
 10  recommendations  20905 non-null  float64
 11  bookstores       20905 non-null  object 
 12  synopsis         20061 non-null  object 
dtypes: float64(1), int64(2), object(10)
memory usage: 2.1+ MB


## Missing Values

In [6]:
df.isna().sum()

title                  0
price                  0
author              1282
publisher              0
isbn                   0
category              19
translator         13366
pages                993
date                   0
series              4927
recommendations      109
bookstores           109
synopsis             953
dtype: int64

In [7]:
df.drop(columns=['author','title','translator','series'], inplace=True)

In [8]:
df.dropna(subset=['category', 'bookstores', 'recommendations', 'synopsis'], inplace=True)

In [9]:
df['pages'] = pd.to_numeric(df['pages'],errors='coerce')
df.fillna({'pages': df['pages'].median()}, inplace=True)